In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd /content/drive/My\ Drive/ECE 657 Project

[Errno 2] No such file or directory: '/content/drive/My Drive/ECE 657 Project'
/content


In [ ]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
data = pd.read_csv('sentiment140.csv', encoding =DATASET_ENCODING , names=DATASET_COLUMNS)

In [ ]:
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
nltk.download('stopwords')
# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data.text = data.text.apply(lambda x: preprocess(x))

In [ ]:
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,awww bummer shoulda got david carr third day
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset update facebook texting might cry result...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,dived many times ball managed save 50 rest go ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole body feels itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,behaving mad see


In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)
data_val, data_test = train_test_split(data_test, test_size = 0.5, random_state = 42)
print("TRAIN size:", len(data_train))
print("VAL size:", len(data_val))
print("TEST size:", len(data_val))

TRAIN size: 1280000
VAL size: 160000
TEST size: 160000


In [ ]:
# vectorizing the document
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(data_train.text)

In [ ]:
# converting the validation and test set to document vectors
X_val = vectorizer.transform(data_val.text)
X_test = vectorizer.transform(data_test.text)

In [ ]:
len(vectorizer.get_feature_names())

290390

In [ ]:
train_text = [txt.split() for txt in data_train.text]

In [ ]:
# word2vec
from gensim.models import Word2Vec
model = Word2Vec(train_text, window = 10, size= 100)

In [ ]:
len(model.wv.vocab)

48146

In [ ]:
model.most_similar('woman')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('women', 0.7021552920341492),
 ('child', 0.6473562717437744),
 ('lady', 0.6356279850006104),
 ('person', 0.6015372276306152),
 ('guy', 0.5993746519088745),
 ('men', 0.592644214630127),
 ('kid', 0.5846248865127563),
 ('young', 0.5845394134521484),
 ('bastard', 0.5623478889465332),
 ('man', 0.5577653050422668)]

In [ ]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(model[model.wv.vocab][0])

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [ ]:
mean_vec_tr = MeanEmbeddingVectorizer(model)
doc_vec = mean_vec_tr.transform(train_text)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [ ]:
doc_vec[0]

array([-0.0283521 ,  0.45632744, -0.37189025, -0.07039715,  0.23196839,
        0.09614218,  0.04733537,  0.41511786, -0.01634319,  0.39866415,
        0.04223465, -0.49799603, -0.58629072,  0.54126114,  0.23038946,
        0.35326102,  0.19724467, -0.03839416,  0.08720204,  0.56486696,
       -0.15509661,  0.37778062, -0.58151716, -0.54122108,  0.18742324,
        0.16811162,  0.17672335,  0.4662357 ,  0.75305569,  0.44258657,
       -0.41324016, -0.09328782, -0.36064625,  0.51544398,  0.93786657,
        0.20313103,  0.04181997, -0.45873091, -0.20437144,  0.28157932,
        0.47849819, -0.59529531,  0.16743304, -0.14271334, -1.13574493,
        0.4291715 , -0.18954045, -0.262135  , -0.06879935, -0.16593626,
        0.59348547,  0.19647311, -0.35657611, -0.0739645 , -0.21431471,
       -0.46862364, -0.22569694,  0.24703847,  0.35155824, -0.26463935,
       -0.35971558, -0.30934036, -0.30264065, -0.26746988, -0.68953741,
        0.19965151,  0.05754243, -0.3335318 ,  0.02172035, -1.16

In [ ]:
doc_vec.shape

(1280000, 100)

In [ ]:
!pip install SimpSOM

  Created wheel for SimpSOM: filename=SimpSOM-1.3.4-cp36-none-any.whl size=17086 sha256=c71d85d2491664f37d805eeacba75d6c256b88e9706e1f4753732a3e032f1ccd
  Stored in directory: /root/.cache/pip/wheels/51/76/ee/c02f0bc20af4cd8f46ee4142ebf7bb654b737d6d8f2360d26b
Successfully built SimpSOM


In [ ]:
import SimpSOM as sps

In [ ]:
raw_data = doc_vec
#Build a network 20x20 with a weights format taken from the raw_data and activate Periodic Boundary Conditions. 
net = sps.somNet(100, 100, raw_data, PBC=True)

#Train the network for 10000 epochs and with initial learning rate of 0.01. 
net.train(0.01, 10000)

#Save the weights to file
net.save('filename_weights')

#Print a map of the network nodes and colour them according to the first feature (column number 0) of the dataset
#and then according to the distance between each node and its neighbours.
net.nodes_graph(colnum=0)
net.diff_graph()

Periodic Boundary Conditions active.
The weights will be initialised randomly.
Training SOM... done!


In [ ]:
y_train = [1 if y == 4 else 0 for y in data_train['target']]
y_train = np.array(y_train)

In [ ]:
#Project the datapoints on the new 2D network map.
net.project(raw_data, labels=y_train)

In [ ]:
#Cluster the datapoints according to the Quality Threshold algorithm.
net.cluster(raw_data, type='qthresh')